# Autogen Group chat architecture 
A simple example of the group chat architecture to solve a task.

Please set your OpenAI API key and AgentOps key in a local .env file for load_dotenv():
```
OPENAI_API_KEY=sk...
AGENTOPS_API_KEY=
```
or set it in the environment explicity using 
```
import os
os.environ["OPENAI_API_KEY"] = "sk..."
os.environ["AGENTOPS_API_KEY"] = ".."
```

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
!pip3 install lxml
!pip3 install arxiv feedparser tabulate


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import agentops

from autogen import ConversableAgent
agentops.init(os.environ.get("AGENTOPS_API_KEY"), default_tags=["autogen-example-2"])

# Ref: https://colab.research.google.com/github/autogenhub/autogen/blob/main/notebook/agentchat_groupchat_research.ipynb

🖇 AgentOps:  WARNING: agentops is out of date. Please update with the command: 'pip install --upgrade agentops'
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=7792b2b2-0e6e-4bf1-9650-1677f04620c2


In [4]:
gpt4_config_list = {"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]}

In [5]:
import autogen 

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config=False,
)
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config_list,
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)


In [8]:
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

In [11]:
planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code.
Explain the plan first. Be clear which step is performed by an engineer and when admin (user) approval is needed.
""",
    llm_config=gpt4_config_list,
)



In [13]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, planner, executor], messages=[], max_round=50
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config_list)

In [14]:
user_proxy.initiate_chat(
    manager,
    message="""
find papers on LLM applications from arxiv in the last 2 months, create a markdown table of different domains.
""",
)

Admin (to chat_manager):


find papers on LLM applications from arxiv in the last 2 months, create a markdown table of different domains.


--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

Plan Explanation:

Step 1 - Engineer:
An engineer will write a script to fetch papers on LLM (Lifetime Learning Machines) applications from arXiv.

Step 2 - Engineer/Admin (User):
The script will filter the fetched papers to include only those published in the last 2 months. The engineer will run the script and generate a list of papers.

Step 3 - Admin Approval:
The admin will review and approve the list of papers.

Step 4 - Engineer:
The engineer will categorize the papers based on different domains (e.g., Machine Learning, Computer Science, etc.). 

Step 5 - Engineer:
The engineer will then write another script to generate a markdown table. This table will include fields such as the title of the paper, authors, arX

Replying as Admin. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  yes


Admin (to chat_manager):

yes

--------------------------------------------------------------------------------

Next speaker: Engineer

Engineer (to chat_manager):

First, we'll need to define what is meant by "LLM applications" in the context of arXiv papers. ArXiv is an online repository of scientific papers and the term "LLM" can have different meanings in different scientific domains. Therefore, I will assume that we are talking about papers that involve "Lifetime Learning Machines". This will be our search query.

We will use arXiv API for Python to fetch the papers. 

Here is a Python script to fetch papers related to "Lifetime Learning Machines" from arXiv in the last 2 months:

```python
from datetime import date, timedelta
import arxiv

# Calculate the date 2 months ago
two_months_ago = date.today() - timedelta(days=60)

# Search for papers
search = arxiv.Search(
  query="Lifetime Learning Machines",
  max_results=1000,
  sort_by=arxiv.SortCriterion.Relevance
)

papers = []
f

Replying as Admin. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Admin (to chat_manager):



--------------------------------------------------------------------------------

Next speaker: Engineer

Engineer (to chat_manager):

Great! We have successfully fetched 20 papers related to "Lifetime Learning Machines" that were published in the last two months on arXiv. 

Now, let's proceed to categorizing these papers based on their domains or categories and creating a markdown table. This table will include fields such as the title of the paper, authors, arXiv link, published date, and domain.

Here is the Python script to generate the markdown table:

```python
# Function to convert data to markdown table format
def to_markdown_table(data, headers):
    table = ' | '.join(headers) + '\n' + ' | '.join('-' * len(h) for h in headers) + '\n'
    for row in data:
        table += ' | '.join(str(row[h]) for h in headers) + '\n'
    return table

# Headers for the table
headers = ['Title', 'Auth

...


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Admin (to chat_manager):



--------------------------------------------------------------------------------

Next speaker: Admin



Replying as Admin. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Admin (to chat_manager):



--------------------------------------------------------------------------------

Next speaker: Admin



Replying as Admin. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


ChatResult(chat_id=None, chat_history=[{'content': '\nfind papers on LLM applications from arxiv in the last 2 months, create a markdown table of different domains.\n', 'role': 'assistant', 'name': 'Admin'}, {'content': 'Plan Explanation:\n\nStep 1 - Engineer:\nAn engineer will write a script to fetch papers on LLM (Lifetime Learning Machines) applications from arXiv.\n\nStep 2 - Engineer/Admin (User):\nThe script will filter the fetched papers to include only those published in the last 2 months. The engineer will run the script and generate a list of papers.\n\nStep 3 - Admin Approval:\nThe admin will review and approve the list of papers.\n\nStep 4 - Engineer:\nThe engineer will categorize the papers based on different domains (e.g., Machine Learning, Computer Science, etc.). \n\nStep 5 - Engineer:\nThe engineer will then write another script to generate a markdown table. This table will include fields such as the title of the paper, authors, arXiv link, and domain.\n\nStep 6 - Admi

Title | Authors | Link | Published | Domains
----- | ------- | ---- | --------- | -------
Sparse Low-Ranked Self-Attention Transformer for Remaining Useful Lifetime Prediction of Optical Fiber Amplifiers | Dominic Schneider, Lutz Rapp | http://arxiv.org/abs/2409.14378v2 | 2024-09-22 09:48:45+00:00 | cs.LG, cs.AI, eess.SP
Comparing Differentiable and Dynamic Ray Tracing: Introducing the Multipath Lifetime Map | Jérome Eertmans, Enrico Maria Vittuci, Vittorio Degli Esposti, Laurent Jacques, Claude Oestges | http://arxiv.org/abs/2410.14535v2 | 2024-10-18 15:23:29+00:00 | eess.SP, cs.LG, 51-08, D.2.2; D.2.8; D.2.13
Energy-Aware Federated Learning in Satellite Constellations | Nasrin Razmi, Bho Matthiesen, Armin Dekorsy, Petar Popovski | http://arxiv.org/abs/2409.14832v1 | 2024-09-23 09:01:17+00:00 | cs.DC, cs.LG, eess.SP
Enhancing Trust in Clinically Significant Prostate Cancer Prediction with Multiple Magnetic Resonance Imaging Modalities | Benjamin Ng, Chi-en Amy Tai, E. Zhixuan Zeng, Alexander Wong | http://arxiv.org/abs/2411.04662v1 | 2024-11-07 12:48:27+00:00 | cs.LG
Compressing Recurrent Neural Networks for FPGA-accelerated Implementation in Fluorescence Lifetime Imaging | Ismail Erbas, Vikas Pandey, Aporva Amarnath, Naigang Wang, Karthik Swaminathan, Stefan T. Radev, Xavier Intes | http://arxiv.org/abs/2410.00948v1 | 2024-10-01 17:23:26+00:00 | eess.IV, cs.LG, q-bio.QM
Geometric Signatures of Compositionality Across a Language Model's Lifetime | Jin Hwa Lee, Thomas Jiralerspong, Lei Yu, Yoshua Bengio, Emily Cheng | http://arxiv.org/abs/2410.01444v2 | 2024-10-02 11:54:06+00:00 | cs.CL, cs.AI, cs.IT, cs.LG, math.IT
SambaMixer: State of Health Prediction of Li-ion Batteries using Mamba State Space Models | José Ignacio Olalde-Verano, Sascha Kirch, Clara Pérez-Molina, Sergio Martin | http://arxiv.org/abs/2411.00233v1 | 2024-10-31 22:12:27+00:00 | cs.LG
NeoPhysIx: An Ultra Fast 3D Physical Simulator as Development Tool for AI Algorithms | Jörn Fischer, Thomas Ihme | http://arxiv.org/abs/2411.05799v1 | 2024-10-26 09:53:07+00:00 | cs.RO, cs.AI, cs.LG
Unlocking Real-Time Fluorescence Lifetime Imaging: Multi-Pixel Parallelism for FPGA-Accelerated Processing | Ismail Erbas, Aporva Amarnath, Vikas Pandey, Karthik Swaminathan, Naigang Wang, Xavier Intes | http://arxiv.org/abs/2410.07364v1 | 2024-10-09 18:24:23+00:00 | physics.optics, cs.AI, cs.DC, cs.LG
Physics-based inverse modeling of battery degradation with Bayesian methods | Micha C. J. Philipp, Yannick Kuhn, Arnulf Latz, Birger Horstmann | http://arxiv.org/abs/2410.19478v1 | 2024-10-25 11:14:30+00:00 | physics.chem-ph, physics.app-ph
Designing tungsten armoured plasma facing components to pulsed heat loads in magnetic fusion machines | R Mitteau, M Diez, M Firdaouss | http://arxiv.org/abs/2410.13269v1 | 2024-10-17 06:44:38+00:00 | physics.plasm-ph
GeoLife+: Large-Scale Simulated Trajectory Datasets Calibrated to the GeoLife Dataset | Hossein Amiri, Richard Yang, Andreas Zufle | http://arxiv.org/abs/2410.11853v1 | 2024-10-02 15:01:13+00:00 | cs.DB, cs.IR, cs.LG, cs.SI
IceCloudNet: 3D reconstruction of cloud ice from Meteosat SEVIRI | Kai Jeggle, Mikolaj Czerkawski, Federico Serva, Bertrand Le Saux, David Neubauer, Ulrike Lohmann | http://arxiv.org/abs/2410.04135v1 | 2024-10-05 12:15:38+00:00 | physics.ao-ph, cs.AI, cs.CV, J.2
The Milky Way Radial Metallicity Gradient as an Equilibrium Phenomenon: Why Old Stars are Metal-Rich | James W. Johnson, David H. Weinberg, Guillermo A. Blanc, Ana Bonaca, Gwen C. Rudie, Yuxi, Lu, Bronwyn Reichardt Chu, Emily J. Griffith, Tawny Sit, Jennifer A. Johnson, Liam O. Dubay, Miqaela K. Weller, Daniel A. Boyea, Jonathan C. Bird | http://arxiv.org/abs/2410.13256v1 | 2024-10-17 06:27:36+00:00 | astro-ph.GA, astro-ph.CO
Inverse Problems and Data Assimilation: A Machine Learning Approach | Eviatar Bach, Ricardo Baptista, Daniel Sanz-Alonso, Andrew Stuart | http://arxiv.org/abs/2410.10523v1 | 2024-10-14 14:01:35+00:00 | stat.ML, cs.LG, math.OC
Symmetry-Enriched Learning: A Category-Theoretic Framework for Robust Machine Learning Models | Ronald Katende | http://arxiv.org/abs/2409.12100v1 | 2024-09-18 16:20:57+00:00 | cs.LG
Quantum Machine Learning: An Interplay Between Quantum Computing and Machine Learning | Jun Qi, Chao-Han Yang, Samuel Yen-Chi Chen, Pin-Yu Chen | http://arxiv.org/abs/2411.09403v1 | 2024-11-14 12:27:50+00:00 | quant-ph, cs.AI
Proactive Fraud Defense: Machine Learning's Evolving Role in Protecting Against Online Fraud | Md Kamrul Hasan Chy | http://arxiv.org/abs/2410.20281v1 | 2024-10-26 21:34:28+00:00 | cs.LG
Identification and Mitigating Bias in Quantum Machine Learning | Nandhini Swaminathan, David Danks | http://arxiv.org/abs/2409.19011v1 | 2024-09-23 21:31:16+00:00 | quant-ph, cs.AI, cs.LG
The State of Julia for Scientific Machine Learning | Edward Berman, Jacob Ginesin | http://arxiv.org/abs/2410.10908v1 | 2024-10-14 01:43:23+00:00 | cs.LG, cs.MS, cs.PL
